# Treibstoffmischung

## Problemstellung

Eine Raffinerie produziert drei Arten von bleifreiem Benzin, die sich in ihrer Oktanzahl (ON) unterscheiden: A: Normalbenzin ($ON \geq 87$), B: Premiumbenzin ($ON \geq 89$) und C: Superbenzin ($ON \geq 92$). Die Raffinierung von Rohöl umfasst drei Komponenten.

1. In der ersten Komponente wird das Rohöl zu einem Ausgangsstoff (feedstock) mit einer Oktanzahl ON = 82 destilliert. Es werden 5 Barrel Rohöl benötigt, um ein Barrel Rohöl zu erzeugen. Die Kapazität der Destillationsanlagen beträgt 1,500 Tausend Barrel pro Tag.
2. Ein Teil des Rohöls wird in einer thermischen Krackanlage weiterverarbeitet, die Benzin mit ON = 98 herstellt. Die Crackanlage produziert ein halbes Barrel Benzin aus einem Barrel Rohöl. Die Kapazität des Crackers ist auf 200,000 Barrel Rohöl pro Tag begrenzt.
3. Das letzte Element in der Raffinerung ist ein Blender, der Benzin aus dem Cracker und den Ausgangsstoff aus der Destillation mischt. Die Oktanzahl der Mischungen ist ungefähr gleich dem gewichteten Durchschnitt der Oktanzahl ihrer Bestandteile. Beispiel: 1 Barrel mit ON=90 und 2 Barrels mit ON=100 ergeben drei Barrels mit Oktanzahl $(1 \times 90 + 2 \times 100) / 3 \approx 96.6$
4. Die Nachfrage nach den drei Benzinsorten ist auf 50, 30 bzw. 40 Tausend Barrels pro Tag begrenzt.
5. Die Raffinerie schätzt den Gewinn pro Barrel Benzin für die drei Benzinsorten auf 6, 7 und 8 EUR.

Konstruieren Sie ein mathematisches Modell, das den optimalen Produktionsplan (mit dem höchsten Gewinn) finden kann. Das Produktionsschema ist in @fig-refinery dargestellt.


```{mermaid}
%%| label: fig-refinery
%%| fig-cap: Das Produktionsschema der Raffinerie

flowchart LR
    A[Destillation 5:1] -->|ON=82| B[Cracker 2:1]
    A --> |ON=82| C
    B -->|ON=98| C[Blender 1:1]
    C --> D(ON=87)
    C --> E(ON=89)
    C --> F(ON=92)
```


In [29]:
%pip install gurobipy

import gurobipy as gp
from gurobipy import GRB
import pandas as pd

products = pd.DataFrame({
    'Product': ['A', 'B', 'C'],
    'Profit': [6, 7, 8],
    'ON': [87, 89, 90],
    'Demand': [50000, 30000, 40000],
}).set_index('Product')

sources = pd.DataFrame({
    'Name': ["Distillation", "Cracker"],    
    'ON': [82, 98],
    'Capacity': [1.5e6, 200e3],
}).set_index('Name')

products

Note: you may need to restart the kernel to use updated packages.


,Profit,ON,Demand
Product,,,
A,6,87,50000
B,7,89,30000
C,8,90,40000


In [11]:
sources

,ON,Capacity
Name,,
Distillation,82,1500000.0
Cracker,98,200000.0


In [31]:
m = gp.Model("Treibstoffmischung")
m.Params.LogToConsole = 0

# Enscheidungsvariablen

x = m.addVars(products.index, sources.index, name="x", vtype=GRB.CONTINUOUS)

# Objective

m.setObjective(gp.quicksum(products.loc[p, 'Profit'] * x[p, s] for p in products.index for s in sources.index), GRB.MAXIMIZE)

m.update()

# Einschränkungen

## Oktanzahlen

m.addConstrs(
    (gp.quicksum(x[p, s] * sources.loc[s, 'ON'] for s in sources.index) >= products.loc[p, 'ON'] * x.sum(p, '*') for p in products.index), 
    name = "Oktanzahl"
)

# Kapazitäten

m.addConstr(2 * x.sum('*', 'Cracker') <= sources.loc['Cracker', 'Capacity'], name="Cracker")
m.addConstr(5 * x.sum('*', 'Distillation') + 5 * 2 * x.sum('*', 'Cracker') <= sources.loc['Distillation', 'Capacity'], name="Distillation")

# Nachfrage

m.addConstrs(
    (gp.quicksum(x[p, s] for s in sources.index) <= products.loc[p, 'Demand'] for p in products.index),
    name="Nachfrage"
)

# Das Modell als LP-Datei speichern und ausgeben

m.write("Treibstoffmischung.lp")

with open("Treibstoffmischung.lp") as f:
    print(f.read())

m.optimize()

Set parameter LogToConsole to value 0
\ Model Treibstoffmischung
\ LP format - for model browsing. Use MPS format to capture full model detail.
Maximize
  6 x[A,Distillation] + 6 x[A,Cracker] + 7 x[B,Distillation]
   + 7 x[B,Cracker] + 8 x[C,Distillation] + 8 x[C,Cracker]
Subject To
 Oktanzahl[A]: - 5 x[A,Distillation] + 11 x[A,Cracker] >= 0
 Oktanzahl[B]: - 7 x[B,Distillation] + 9 x[B,Cracker] >= 0
 Oktanzahl[C]: - 8 x[C,Distillation] + 8 x[C,Cracker] >= 0
 Cracker: 2 x[A,Cracker] + 2 x[B,Cracker] + 2 x[C,Cracker] <= 200000
 Distillation: 5 x[A,Distillation] + 10 x[A,Cracker] + 5 x[B,Distillation]
   + 10 x[B,Cracker] + 5 x[C,Distillation] + 10 x[C,Cracker] <= 1.5e+06
 Nachfrage[A]: x[A,Distillation] + x[A,Cracker] <= 50000
 Nachfrage[B]: x[B,Distillation] + x[B,Cracker] <= 30000
 Nachfrage[C]: x[C,Distillation] + x[C,Cracker] <= 40000
Bounds
End



In [33]:
# Die Lösung des Modells als DataFrame speichern

vars_df = pd.DataFrame({
    'Product': [p for p in products.index for s in sources.index],
    'Source': [s for p in products.index for s in sources.index],
    'Amount': [x[p, s].x for p in products.index for s in sources.index],
})

vars_df

,Product,Source,Amount
0,A,Distillation,0.0
1,A,Cracker,50000.0
2,B,Distillation,16875.0
3,B,Cracker,13125.0
4,C,Distillation,20000.0
5,C,Cracker,20000.0
